# GraphQA using LangChain and Kùzu
In this notebook, we will use the LangChain library to answer questions on top of the Kùzu graph we
just created in the previous section. The example below uses the OpenAI GPT-3.5 turbo model to generate
Cypher and answer questions via a text-to-Cypher pipeline, but you can use any other model and see
how it performs.

We start by opening a connection to the existing database and loading the `OPENAI_API_KEY` variable
from a local `.env` file.

In [1]:
# !uv pip install python-dotenv langchain langchain-community langchain-openai

In [2]:
import os

import kuzu
from dotenv import load_dotenv

# Load OpenAI API key from .env file
load_dotenv()
assert "OPENAI_API_KEY" in os.environ, "Please set OPENAI_API_KEY in the .env file"

db = kuzu.Database("../ex_db_kuzu")
conn = kuzu.Connection(db)

In [3]:
from langchain.chains import KuzuQAChain
from langchain_community.graphs import KuzuGraph
from langchain_openai import ChatOpenAI

In [4]:
# Create a graph object for KuzuQAChain and print the schema
graph = KuzuGraph(db)
print(graph.get_schema)

Node properties: [{'properties': [('id', 'INT64'), ('account_id', 'STRING'), ('balance', 'DOUBLE'), ('betweenness_centrality', 'DOUBLE')], 'label': 'Account'}, {'properties': [('address', 'STRING')], 'label': 'Address'}, {'properties': [('id', 'INT64'), ('name', 'STRING'), ('state', 'STRING'), ('zip', 'INT64'), ('email', 'STRING')], 'label': 'Person'}]
Relationships properties: [{'properties': [], 'label': 'Owns'}, {'properties': [], 'label': 'LivesIn'}, {'properties': [('amount', 'DOUBLE')], 'label': 'Transfer'}]
Relationships: ['(:Person)-[:Owns]->(:Account)', '(:Person)-[:LivesIn]->(:Address)', '(:Account)-[:Transfer]->(:Account)']



This schema is passed as part of the prompt to the LLM, which is then used to generate the Cypher query.
The following example shows how to the GPT-3.5 turbo model is used for both text-to-Cypher and for
answer generation.

In [5]:
chain = KuzuQAChain.from_llm(
    llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0, api_key=os.environ.get("OPENAI_API_KEY")),
    graph=graph,
    verbose=True,
)


We can then ask questions in natural language.

In [6]:
chain.invoke("What is the highest amount transferred in this dataset?")



> Entering new KuzuQAChain chain...
Generated Cypher:
MATCH ()-[:Transfer {amount: $highest_amount}]->()
RETURN $highest_amount
Full Context:
[]

> Finished chain.


{'query': 'What is the highest amount transferred in this dataset?',
 'result': "I don't know the answer."}

In [7]:
chain.invoke("Is there a path up to a depth of 3 between the persons named Karissa and Anna?")



> Entering new KuzuQAChain chain...
Generated Cypher:
MATCH path = (:Person {name: 'Karissa'})-[*1..3]-(:Person {name: 'Anna'})
RETURN path
Full Context:
[{'path': {'_nodes': [{'_id': {'offset': 6, 'table': 0}, '_label': 'Person', 'id': 7, 'name': 'Karissa', 'state': 'FL', 'zip': 32325, 'email': 'kpnichols_malone@gmail.com', 'address': None, 'account_id': None, 'balance': None, 'betweenness_centrality': None}, {'_id': {'offset': 6, 'table': 2}, '_label': 'Account', 'id': 7, 'name': None, 'state': None, 'zip': None, 'email': None, 'address': None, 'account_id': '120747146', 'balance': 7354.0, 'betweenness_centrality': 0.0}, {'_id': {'offset': 14, 'table': 2}, '_label': 'Account', 'id': 15, 'name': None, 'state': None, 'zip': None, 'email': None, 'address': None, 'account_id': '073343503', 'balance': 8048.0, 'betweenness_centrality': 0.006535947712418301}, {'_id': {'offset': 14, 'table': 0}, '_label': 'Person', 'id': 15, 'name': 'Anna', 'state': 'MN', 'zip': 55084, 'email': 'amrowland@

{'query': 'Is there a path up to a depth of 3 between the persons named Karissa and Anna?',
 'result': 'Yes, there is a path up to a depth of 3 between the persons named Karissa and Anna.'}

## Experiment with open source LLMs
You can use different LLMs, including open source ones, for text-to-Cypher and the answer generation stages. See the
[LangChain docs](https://python.langchain.com/v0.2/docs/integrations/graphs/kuzu_db/#use-separate-llms-for-cypher-and-answer-generation)
for such an example.

Open source LLMs can be self-hosted and served on a local endpoint. In
this example, we use a _much_ cheaper locally running `Mistral-7B-OpenOrca-GGUF` model from LMStudio
for text-to-Cypher, followed by OpenAI's GPT-3.5 turbo model for answer generation. We are still able
to call the `ChatOpenAI` class in both cases because LMStudio's local server mimics OpenAI's API endpoints.

Note that cheap and small open source LLMs may not perform as well as the proprietary, general-purpose ones,
so to obtain best performance on Cypher generation (as well as inference from Cypher quuery results),
you may need to fine-tune a more powerful model.

In [8]:
chain = KuzuQAChain.from_llm(
    qa_llm=ChatOpenAI(base_url="http://localhost:1234/v1", temperature=0, api_key="not_needed"),
    cypher_llm=ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0, api_key=os.environ.get("OPENAI_API_KEY")),
    graph=graph,
    verbose=True,
)

In [9]:
chain.invoke("Which addresses have the most people living in them? Return only the top 3.")



> Entering new KuzuQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:LivesIn]->(a:Address)
RETURN a, COUNT(p) AS numPeople
ORDER BY numPeople DESC
LIMIT 3
Full Context:
[{'a': {'_id': {'offset': 1, 'table': 1}, '_label': 'Address', 'address': '953 Kyle Loop, Johnview, MT 59065'}, 'numPeople': 3}, {'a': {'_id': {'offset': 2, 'table': 1}, '_label': 'Address', 'address': '1919 Joshua Square Suite 643, Carmen Town, WY 83111'}, 'numPeople': 2}, {'a': {'_id': {'offset': 4, 'table': 1}, '_label': 'Address', 'address': '829 Zimmerman Points Suite 688, Port Eduardo, FL 32325'}, 'numPeople': 2}]

> Finished chain.


{'query': 'Which addresses have the most people living in them? Return only the top 3.',
 'result': ' The address with the most people living in it is "953 Kyle Loop, Johnview, MT 59065" with 3 people. The second address with the most people is "1919 Joshua Square Suite 643, Carmen Town, WY 83111" with 2 people. The third address with the most people is "829 Zimmerman Points Suite 688, Port Eduardo, FL 32325" also with 2 people.'}

## Next steps
Feel free to experiment with other LLMs and see how they perform on your own data. As the natural
language questions become more complex, it might result in incorrect Cypher generation, no matter
how good the underlying LLM. In such cases, a query rewriting step may be required to provide better
context to the cypher-generating LLM.

This notebook is just the starting point of utilizing knowledge graphs for retrieval and QA tasks. You can
look at more advanced pipelines that utilize agents, memory and routers via the LangChain and LlamaIndex frameworks,
or simply roll out your own abstractions that fit your use case.

Have fun using graphs, and `pip install kuzu`!